<a href="https://colab.research.google.com/github/ThomasKismarton/Uniform_project/blob/main/Uniform_Assignor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import json
import numpy as np
import pandas as pd
import unittest

In [28]:
# Reads in a DataFrame and converts sizes to decimals
def size_conv(items):
  size_dict = {
    'YS': 1,
    'YM': 2,
    'YL': 3.5,
    'YXL': 3.75,
    'AXS': 3.5,
    'AS': 4,
    'AM': 5,
    'AL': 6,
    'AXL': 7,
    'A2XL': 8
  }
  items = items.replace({'Size': size_dict})
  return items

In [29]:
def jmatch(data, people):
  tol = 0
  while (tol <= 4):
    for player in people.iterrows(): # For each player
      size_diff = data[1] - player[1][1] # Find the difference between player size and uniform size
      if (abs(size_diff) <= tol): # If close enough <<and not data[0] in final_nums.values()>> - removes dupes
        final_nums[player[1][0]] = data[0] # Add it to the roster of numbers
        final_nums['Mismatch Score'] += abs(size_diff) # Track overall mismatch score
        return people.drop(player[0]) # Remove the player from the list of players needing jerseys
    tol += 0.25
  return people # Case to catch a uniform without a match

In [30]:
# Begins matching uniforms to players in prio of least available sizes first.
def smatch(sizelist, ps):
  for size in sizelist: # For all of the sizes of uniforms available,
    for uniform in sizedf[size].iterrows(): # For each uniform of that size,
      unidata = uniform[1] # Grab the uniform size & number
      ps = jmatch(unidata, ps) # Match the uniform to a player and remove from list if possible
  return ps

In [32]:
def uniform_full_match(inv, roster):
  final_nums.clear() # Removes previous team numbers, resets MMS
  sizedf.clear() # Resets the size dataframe as well
  final_nums['Mismatch Score'] = 0
  players = assign(inv, roster)
  return final_nums if players.empty else {'Mismatch Score': np.nan}

In [34]:
def assign(inv_file, player_file):
  inventory = size_conv(pd.read_csv(inv_file))
  players = size_conv(pd.read_csv(player_file))

  for size in inventory['Size'].unique():
    sizedf[size] = inventory.loc[inventory['Size'] == size]
  lp = sorted(sizedf, key = lambda x:sizedf[x].shape[0]) # Holds the list of all sizes sorted in ascending qty available

  # As players are assigned uniforms, remove the uniforms from the inventory
  # Once all the players have been assigned a uniform, then return the list of names -> numbers
  players = smatch(lp, players)
  return players

In [15]:
def erasure_gen(arr):
  erasure_arr = np.copy(arr)
  row_vector = np.nansum(arr, axis=1)
  col_vector = np.nansum(arr, axis=0)
  for row in range(arr.shape[0]):
    for col in range(arr.shape[1]):
      erasure_arr[row][col] = np.nansum([row_vector[row], col_vector[col], 2*arr[row][col]]) * arr[row][col]
  return erasure_arr

In [53]:
def populate(inv, ros):
  assignment_dict = {}
  uni_dict = {}
  for uni_bin in os.scandir(inv):
    uni_dict[uni_bin.name] = {}
    # Iterates through all teams
    for team in os.scandir(rosters):
      # Assigns uniform numbers to players, determines the mismatch score (MMS) per team for a specific bin
      team_nums = uniform_full_match(uni_bin, team)
      # Records the MMS indexed at (Team, Bin)
      uni_dict[uni_bin.name][team.name] = team_nums['Mismatch Score']
      assignment_dict[uni_bin.name + " " + team.name] = dict(team_nums)
  uni_matrix = pd.DataFrame(uni_dict)
  return uni_matrix, assignment_dict

In [52]:
def allocNums(mms, teamlist, binlist):
  combolist = []
  # Begin numpy work with matrices
  np_matrix = mms.to_numpy()
  erasures = erasure_gen(np_matrix)
  while erasures.size > 0:
    index = np.unravel_index(np.argmax(erasures), np.shape(erasures))
    erasures = np.delete(np.delete(erasures, index[0], 0), index[1], 1)

    # Select the team-bin combination with the largest erasure score, remove it from the list, print the numbers
    team = teamlist.pop(index[0])
    ubin = binlist.pop(index[1])
    combolist.append(ubin + " " + team)
  return combolist


In [37]:
# These only ever run once, to initialize the variables.
final_nums = {}
sizedf = {}
path = os.getcwd()
inventory = path + '/Uniforms'
rosters = path + '/Teams'

In [57]:
# Generates a matrix of MMS for bin-team pairings
u_matrix, ad = populate(inventory, rosters)
# team_idx = list(u_matrix.index)
# bin_idx = list(u_matrix.columns)
toPrint = allocNums(u_matrix, list(u_matrix.index), list(u_matrix.columns))
for entry in toPrint:
  el = entry.split(' ')
  print("Team name:", el[1], "| Bin Name:", el[0])
  print(json.dumps(ad[entry], indent = 2))

Team name: B06.csv | Bin Name: Bin1.csv
{
  "Mismatch Score": 14,
  "Ayden Rieke-Wey": 25,
  "Carlos Abram": 2,
  "Dalen Hamilton": 4,
  "Dylan Chung": 46,
  "Edward Brown": 24,
  "Drew Clark-Bolt": 8,
  "Jay Piper": 11,
  "Kensei Sugitani": 14,
  "Lunden Fenster": 16,
  "Mason Duarte": 17,
  "Lucas Hogben": 3,
  "Wyatt Stevens": 5,
  "Chase Larson": 6,
  "Lukas Shriner": 7,
  "Oscar Castillo": 10,
  "Riley Cotton Lopez": 12,
  "Thero Mitchell": 13,
  "Uriel Villegas-Gallegos": 15
}
